<a href="https://colab.research.google.com/github/paulyu8868/test/blob/main/%EC%B9%A8%EB%AA%B0%EB%B0%A9%EC%A7%80%EB%B2%95v2_2(%EC%88%98%EC%88%98%EB%A3%8C%EC%A0%81%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass


import warnings
# 모든 FutureWarning 무시
warnings.filterwarnings('ignore', category=FutureWarning)
# UserWarning도 무시
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
#기간별 주가 데이터 불러오기
def get_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df = df.drop(columns=['Volume', 'Adj Close'])
    df.index = df.index.date

    # 호가 단위 0.01$ 적용
    for col in ['Open', 'High', 'Low', 'Close']: # 시가,고가,저가,종가
        df[col] = df[col].map(round_half_up_to_two)

    # 등락율 계산
    df['Return'] = df['Close'].pct_change()
    df['Return'] = df['Return'].map(pointTopercent)
    #df['Return'] = df['Return'].map(round_half_up_to_two)

    return df

In [3]:
# 소수 셋째자리에서 반올림
def round_half_up_to_two(num):
    try:
        if isinstance(num, (float, int)):
            num_100 = num * 100
            if num_100 - int(num_100) >= 0.5:
                return (int(num_100) + 1) / 100
            else:
                return int(num_100) / 100
        else:
            return num
    except:
        return num

# 퍼센트로 변환
def pointTopercent(num):
    return round_half_up_to_two(num*100)

In [4]:
def calculate_mdd(equity_curve):
    """MDD(Maximum Drawdown) 계산"""
    cummax = equity_curve.cummax()
    drawdown = (equity_curve - cummax) / cummax * 100
    mdd = drawdown.min()
    return mdd

In [46]:
# 침몰방지법 매매로직
def prevent_drown_down_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period,welfare, fee):
    funds = initial_funds # 초기 자금
    one_buy_amount = initial_funds / buy_portion # 회차별 매수금액
    holdings = 0 # 보유 주식 수
    buy_records = []  # 각 매수 건을 저장하여 관리 (개별 매도 관리)
    trade_history = []  # 매도 기록 저장
    trade_id = 1  # 매수 회차별 ID (매수 구분)
    fee = fee/100
    total_fee = 0

    T=0 # T값 = 보유 회차 수

    # 시작일 - 종료일 시뮬레이션 진행
    for i in range(start_idx, start_idx + simulation_period + 1):
        # 데이터 시리즈로 가져오기
        current_date = df.index[i]
        open_price = float(df['Open'].iloc[i]) #시가
        high_price = float(df['High'].iloc[i]) #고가
        close_price = float(df['Close'].iloc[i]) #종가
        return_val = float(df['Return'].iloc[i]) #등락율
        prev_price = float(df['Close'].iloc[i-1]) if i > 0 else close_price #전날 종가


        # 오늘 데이터
        df_res.at[i, '날짜'] = current_date
        df_res.at[i, '시가'] = open_price
        df_res.at[i, '고가'] = high_price
        df_res.at[i, '종가'] = close_price
        df_res.at[i, '등락율'] = f"{round_half_up_to_two(return_val)}%"

        # 당일 종가
        price = close_price

        # 매수/매도 칼럼 초기화
        df_res.at[i, 'LOC 매수'] = 0
        df_res.at[i, '수익 실현 매도'] = 0
        df_res.at[i, 'MOC 손절'] = 0

        '''
        V2
        매매로직 (*T값 6부터 안전모드)
        매수 : (6-maximumT)%
        매도 : (12.5-2*maximumT)%
        최대보유기간 : (30-3*maximumT)일
        '''
        maximumT=min(T,6) # T값 최대 5까지 적용
        start_T=T # 매매체결전 기준 T값 (해당일에 실제 적용되는 T값)
        loc_buy = (1.06-0.01*maximumT) # 매수 기준
        loc_sell = (1.125-0.02*maximumT) # 매도 기준
        '''
        *복리*
        복리적용 회차금액 = 예수금/(분할수-T)
        '''
        one_buy_welfare = funds / (buy_portion-start_T) if 9>=start_T else funds # 복리
        #one_buy_welfare = funds / 6 #if start_T < 6 else one_buy_amount  # 복리
        # 매수 로직

        if price <= prev_price*loc_buy: # 매수 기준
            if welfare: # 복리적용
                qty = int(one_buy_welfare / (prev_price * loc_buy)) # 복리 적용된 금액만큼 매수
            else: # 복리적용 X
                qty = int(one_buy_amount / (prev_price * loc_buy)) # 원금 10분할
            if funds >= (qty * price)*(1+fee):
                holdings += qty
                funds -= qty * price
                funds -= (qty * price) * fee # 수수료 차감
                total_fee += (qty * price) * fee # 총 수수료 계산
                T +=1 # 회차수 +=1
                buy_records.append({
                    'id': trade_id,
                    'buy_date': current_date,
                    'buy_price': price,
                    'quantity': qty,
                    'days': 0,
                    'type': 'LOC 매수'
                })
                df_res.at[i, 'LOC 매수'] = qty
                trade_id += 1

        # 매도 로직
        if holdings > 0:
            new_buy_records = []  # 매도되지 않은 매수 건을 저장할 새 리스트
            total_sell = 0  # 당일 총 매도 수량

            for record in buy_records: # 보유 주식 순회
                record['days'] += 1  # 보유일 count
                # 각 매수 건별 매도 목표가
                sell_price = record['buy_price'] * loc_sell # 매도 기준

                if price >= sell_price:  # 수익 실현 매도 조건 충족
                    funds += record['quantity'] * price
                    funds -= (record['quantity'] * price) * fee # 수수료 차감
                    total_fee += (record['quantity'] * price) * fee # 총 수수료 계산
                    total_sell += record['quantity']
                    holdings -= record['quantity']
                    T-=1 # 회차수 -=1

                    # 거래 기록 저장
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round_half_up_to_two((price/record['buy_price'] - 1) * 100),
                        '적용 모드': "투자모드" if (maximumT)<6 else "회복모드",
                        '적용 T':start_T
                    })
                elif record['days'] >= (30-maximumT*3):  # 최대 보유일수 경과시 MOC 매도
                    funds += record['quantity'] * price
                    funds -= (record['quantity'] * price) * fee # 수수료 차감
                    total_fee += record['quantity'] * price * fee # 총 수수료 계산
                    df_res.at[i, 'MOC 손절'] = record['quantity']
                    holdings -= record['quantity']
                    T-=1 # 회차수 -=1
                    # 손절 거래 기록 저장
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': (price/record['buy_price'] - 1) * 100,
                        '적용 모드': "투자모드" if (maximumT)<6 else "회복모드",
                        '적용 T':start_T
                    })
                else:
                    new_buy_records.append(record)

            if total_sell > 0:
                df_res.at[i, '수익 실현 매도'] = total_sell

            buy_records = new_buy_records

        # 포트폴리오 상태 저장
        df_res.at[i, '보유 주식 수'] = holdings
        df_res.at[i, '예수금'] = funds
        df_res.at[i, '총 평가액'] = funds + (price * holdings)
        df_res.at[i, '수익율(%)'] = ((funds + (price * holdings)) / initial_funds - 1) * 100
        df_res.at[i, 'MDD'] = calculate_mdd(df_res['총 평가액'])
        df_res.at[i, 'T값'] = T
        df_res.at[i, '모드'] = "회복" if T>=6 else "투자"
        df_res.at[i, '총 수수료($)'] = round_half_up_to_two(total_fee)

    final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period]))
    final_mdd = calculate_mdd(df_res['총 평가액'])
    return round_half_up_to_two((final_value / initial_funds - 1) * 100), df_res, final_value, pd.DataFrame(trade_history) , final_mdd, total_fee

In [50]:
if __name__ == "__main__":
    start_date = '2020-01-01'
    end_date = '2020-12-31'
    initial_funds = 100000 # 초기 자금
    buy_portion = 10  # 회차 분할 수
    welfare = False # 복리적용X
    #welfare = True # 복리적용
    fee = 0.044 # 수수료(%)

    # 시작일
    start_date_dt = datetime.strptime(start_date,'%Y-%m-%d')
    start_date_before_30 = start_date_dt - timedelta(days=30) # 시작일 30일 전
    start_date_before_30 = start_date_before_30.strftime('%Y-%m-%d')

    # 종료일 다음 날 계산
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
    next_day = end_date_dt + timedelta(days=1)
    end_day_next = next_day.strftime('%Y-%m-%d')

    # ticker 값으로 종목 선택 가능 ex) TQQQ
    df = get_data(ticker='TQQQ', start=start_date_before_30, end=end_day_next) # 시작일 30일 전 데이터부터 가져오기
    df_length = len(df) - len(get_data(ticker='TQQQ', start=start_date, end=end_day_next))
    df_res = pd.DataFrame(columns=['날짜', '시가', '고가', '종가', '등락율', 'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                  '보유 주식 수', '예수금', '총 평가액', '수익율(%)', 'MDD','T값',"모드",'총 수수료($)'])

    # 시뮬레이션 실행
    return_rate, df_res, final_value, df_trades ,final_mdd, total_fee= prevent_drown_down_simulation(
        df, df_res, initial_funds, buy_portion, df_length, len(df)-1-df_length, welfare , fee)

    # 매매 통계 출력
    print('\n' + '='*80)
    print("매매 통계")
    print('='*80)
    print(f"총 매매 횟수: {len(df_trades)} 회")
    print(f"평균 보유기간: {df_trades['보유기간'].mean():.1f} 일")
    print(f"평균 수익률: {df_trades['수익률(%)'].mean():.2f}%")
    win_rate = len(df_trades[df_trades['수익률(%)'] > 0]) / len(df_trades) * 100
    print(f"승률: {win_rate:.2f}%")

    print('\n' + '='*80)
    print(f"{start_date} ~ {end_date} 동안의 자산 변동 결과")
    print('='*80)
    print(f"최초 보유 금액: ${initial_funds:,.2f}")
    print(f"최종 보유 금액: ${final_value:,.2f}")
    print(f"원금 변화율: {return_rate}%")
    print(f"MDD : {final_mdd:.2f}%")
    print(f"총 수수료 : ${total_fee:.2f}")
    print('='*80)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



매매 통계
총 매매 횟수: 161 회
평균 보유기간: 6.1 일
평균 수익률: 3.63%
승률: 88.82%

2020-01-01 ~ 2020-12-31 동안의 자산 변동 결과
최초 보유 금액: $100,000.00
최종 보유 금액: $153,545.38
원금 변화율: 53.55%
MDD : -23.07%
총 수수료 : $1378.84


In [51]:
# 날짜별 거래 현황 출력
print("\n[일별 거래 현황]")
df_res_style = df_res.style.format({
    '시가': '{:.2f}',
    '고가': '{:.2f}',
    '종가': '{:.2f}',
    'LOC 매수': '{:.0f}',
    '수익 실현 매도': '{:.0f}',
    'MOC 손절': '{:.0f}',
    '보유 주식 수': '{:.0f}',
    '예수금': '{:.2f}',
    '총 평가액': '{:.2f}',
    '수익율(%)': '{:.2f}',
    'MDD': '{:.2f}',
    'T값': '{:.0f}',
    '총 수수료($)': '{:.2f}'
}).set_properties(**{'text-align': 'center'})
display(df_res_style)


[일별 거래 현황]


,날짜,시가,고가,종가,등락율,LOC 매수,수익 실현 매도,MOC 손절,보유 주식 수,예수금,총 평가액,수익율(%),MDD,T값,모드,총 수수료($)
21,2020-01-02,22.18,22.71,22.71,4.94%,435,0,0,435,90116.80,99995.65,-0.00,0.00,1,투자,4.35
22,2020-01-03,21.82,22.50,22.09,-2.73%,419,0,0,854,80857.02,99721.88,-0.28,-0.27,2,투자,8.42
23,2020-01-06,21.57,22.52,22.51,1.9%,435,0,0,1289,71060.86,100076.25,0.08,-0.27,3,투자,12.73
24,2020-01-07,22.53,22.69,22.49,-0.08%,431,0,0,1720,61363.41,100046.21,0.05,-0.27,4,투자,16.99
25,2020-01-08,22.49,23.32,23.00,2.27%,0,0,0,1720,61363.41,100923.41,0.92,-0.27,4,투자,16.99
26,2020-01-09,23.56,23.72,23.58,2.52%,0,1285,0,435,91650.38,101907.68,1.91,-0.27,1,투자,30.32
27,2020-01-10,23.85,23.87,23.39,-0.8%,403,0,0,838,82220.06,101820.88,1.82,-0.27,2,투자,34.47
28,2020-01-13,23.68,24.22,24.21,3.51%,411,0,0,1249,72265.37,102503.66,2.50,-0.27,3,투자,38.85
29,2020-01-14,24.15,24.29,23.92,-1.19%,401,0,0,1650,62669.23,102137.23,2.14,-0.36,4,투자,43.07
30,2020-01-15,23.94,24.28,23.94,0.08%,409,435,0,1624,63282.78,102161.34,2.16,-0.36,4,투자,51.96


In [16]:
# 모든 매매 로그 출력
if not df_trades.empty:
    print("\n[매매 기록]")
    df_trades_style = df_trades.style.format({
        '매수가': '{:.2f}',
        '매수수량': '{:.0f}',
        '매도가': '{:.2f}',
        '매도수량': '{:.0f}',
        '수익률(%)': '{:.2f}',
        '적용 모드': '{:}',
        '적용 T': '{:.0f}'
    }).set_properties(**{'text-align': 'center'})

    # 수익률에 따른 색상 적용
    def color_returns(val):
        color = 'red' if val < 0 else ('blue' if val > 0 else 'black')
        return f'color: {color}'
    def mode_color(val):
        color = 'green' if val=="회복모드" else'black'
        return f'color: {color}'

    df_trades_style = df_trades_style.map(color_returns, subset=['수익률(%)'])
    df_trades_style = df_trades_style.map(mode_color, subset=['적용 모드'])
    display(df_trades_style)

    # 매매 통계 출력
    print('\n' + '='*80)
    print("매매 통계")
    print('='*80)
    print(f"총 매매 횟수: {len(df_trades)} 회")
    print(f"평균 보유기간: {df_trades['보유기간'].mean():.1f} 일")
    print(f"평균 수익률: {df_trades['수익률(%)'].mean():.2f}%")
    win_rate = len(df_trades[df_trades['수익률(%)'] > 0]) / len(df_trades) * 100
    print(f"승률: {win_rate:.2f}%")

    print('\n' + '='*80)
    print(f"{start_date} ~ {end_date} 동안의 자산 변동 결과")
    print('='*80)
    print(f"최초 보유 금액: ${initial_funds:,.2f}")
    print(f"최종 보유 금액: ${final_value:,.2f}")
    print(f"원금 변화율: {return_rate}%")
    print('='*80)


[매매 기록]


,회차,매수일,매수가,매수수량,매도일,매도가,매도수량,보유기간,수익률(%),적용 모드,적용 T
0,2,2020-01-03,22.09,419,2020-01-09,23.58,419,5,6.75,투자모드,4
1,3,2020-01-06,22.51,435,2020-01-09,23.58,435,4,4.75,투자모드,4
2,4,2020-01-07,22.49,431,2020-01-09,23.58,431,3,4.85,투자모드,4
3,1,2020-01-02,22.71,415,2020-01-15,23.94,415,10,5.42,투자모드,4
4,5,2020-01-10,23.39,403,2020-01-16,24.61,403,5,5.22,투자모드,4
5,6,2020-01-13,24.21,411,2020-01-22,25.13,411,7,3.80,투자모드,5
6,7,2020-01-14,23.92,401,2020-01-22,25.13,401,6,5.06,투자모드,5
7,8,2020-01-15,23.94,409,2020-01-22,25.13,409,5,4.97,투자모드,5
8,14,2020-01-27,23.18,400,2020-01-28,24.26,400,2,4.66,회복모드,6
9,15,2020-01-29,24.34,408,2020-01-30,24.62,408,2,1.15,회복모드,6



매매 통계
총 매매 횟수: 161 회
평균 보유기간: 6.1 일
평균 수익률: 3.63%
승률: 88.82%

2020-01-01 ~ 2020-12-31 동안의 자산 변동 결과
최초 보유 금액: $100,000.00
최종 보유 금액: $154,761.78
원금 변화율: 54.76%
